## KNN imputation

**The missing values** are estimated as the **average value from the closest K neighbours**. [KNNImputer from sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html#sklearn.impute.KNNImputer): **Same K will be used** to impute all variables! Can't really optimise K to better predict the missing values! Could optimise K to better predict the target! **Note** If what we want is to predict, **as accurately as possible** the values of the missing data, then, we would not use the KNN imputer, we would build individual KNN algorithms to predict 1 variable from the remaining ones. This is a common regression problem.

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer  # multivariate imputation

## Load data

**List with numerical varables!**

In [14]:
cols_to_use = [
    'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 
    'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 
    'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 
    'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
    'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 
    'GarageCars', 'GarageArea', 'WoodDeckSF',  'OpenPorchSF', 'EnclosedPorch', 
    '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']

**Load the dataset with a selected variables!**

In [20]:
data = pd.read_csv('HousingPrices_train.csv', usecols=cols_to_use)
data.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,548,0,61,0,0,0,0,0,2,2008
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,460,298,0,0,0,0,0,0,5,2007
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,608,0,42,0,0,0,0,0,9,2008
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,642,0,35,272,0,0,0,0,2,2006
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,836,192,84,0,0,0,0,0,12,2008


In [19]:
for var in data.columns:  # find variables with missing data
    if data[var].isnull().sum() > 1:
        print(var, data[var].isnull().sum())

LotFrontage 259
MasVnrArea 8
GarageYrBlt 81


In [16]:
cols_to_use.remove('SalePrice')
X_train, X_test, y_train, y_test = train_test_split(
    data[cols_to_use],
    data['SalePrice'],
    test_size=0.3,
    random_state=0)
X_train.shape, X_test.shape

((1022, 36), (438, 36))

**Reset index, so we can compare values later!**

In [17]:
X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)

## KNN imputation

In [6]:
imputer = KNNImputer(
    n_neighbors=5, # the number of neighbours K
    weights='distance', # the weighting factor
    metric='nan_euclidean', # the metric to find the neighbours
    add_indicator=False,)  # whether to add a missing indicator

In [7]:
imputer.fit(X_train)

KNNImputer(weights='distance')

**Sklearn returns a Numpy array! Make a dataframe!**

In [21]:
train_t = imputer.transform(X_train)
test_t = imputer.transform(X_test)
train_t = pd.DataFrame(train_t, columns=X_train.columns)
test_t = pd.DataFrame(test_t, columns=X_test.columns)
train_t.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60.0,70.115142,9375.0,7.0,5.0,1997.0,1998.0,573.0,739.0,0.0,...,645.0,576.0,36.0,0.0,0.0,0.0,0.0,0.0,2.0,2009.0
1,120.0,42.533053,2887.0,6.0,5.0,1996.0,1997.0,0.0,1003.0,0.0,...,431.0,307.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,2008.0
2,20.0,50.000000,7207.0,5.0,7.0,1958.0,2008.0,0.0,696.0,0.0,...,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2010.0
3,50.0,60.000000,9060.0,6.0,5.0,1939.0,1950.0,0.0,204.0,0.0,...,280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,2009.0
4,30.0,60.000000,8400.0,2.0,5.0,1920.0,1950.0,0.0,290.0,0.0,...,246.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2009.0


**Variables without NA after the imputation!**

In [22]:
train_t[['LotFrontage', 'MasVnrArea', 'GarageYrBlt']].isnull().sum()

LotFrontage    0
MasVnrArea     0
GarageYrBlt    0
dtype: int64

**The obseravtions with NA in the original train set!**

In [23]:
X_train[X_train['MasVnrArea'].isnull()]['MasVnrArea']

420   NaN
490   NaN
642   NaN
824   NaN
921   NaN
Name: MasVnrArea, dtype: float64

**The replacement values in the transformed dataset!**

In [24]:
train_t[X_train['MasVnrArea'].isnull()]['MasVnrArea']

420     99.765717
490     34.106592
642      0.000000
824    375.749332
921     85.817715
Name: MasVnrArea, dtype: float64

**The mean value of the variable (i.e., for mean imputation)!**

In [25]:
X_train['MasVnrArea'].mean()

103.55358898721731

In some cases, the imputation values are **very different from the mean value** we would have used in MeanMedianImputation.

## Imputing a slice of the dataframe

We can use **Feature-engine** to apply the KNNImputer to **a slice of the dataframe.**

In [28]:
from feature_engine.wrappers import SklearnTransformerWrapper

In [29]:
data = pd.read_csv('HousingPrices_train.csv')
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('SalePrice', axis=1),
    data['SalePrice'],
    test_size=0.3,
    random_state=0)
X_train.shape, X_test.shape

((1022, 80), (438, 80))

**Start the KNNimputer inside the SKlearnTransformerWrapper!**

In [30]:
imputer = SklearnTransformerWrapper(
    transformer = KNNImputer(weights='distance'),
    variables = cols_to_use,)

**Fit the wrapper + KNNImputer! Transform the data! Feature-engine returns a dataframe!**

In [32]:
imputer.fit(X_train)
train_t = imputer.transform(X_train)
test_t = imputer.transform(X_test)
train_t.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
64,65,60.0,RL,70.115142,9375.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,GdPrv,NaN,0.0,2.0,2009.0,WD,Normal
682,683,120.0,RL,42.533053,2887.0,Pave,NaN,Reg,HLS,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,11.0,2008.0,WD,Normal
960,961,20.0,RL,50.000000,7207.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2010.0,WD,Normal
1384,1385,50.0,RL,60.000000,9060.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,MnPrv,NaN,0.0,10.0,2009.0,WD,Normal
1100,1101,30.0,RL,60.000000,8400.0,Pave,NaN,Reg,Bnk,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,1.0,2009.0,WD,Normal


**No NA after the imputation!**

In [33]:
train_t['MasVnrArea'].isnull().sum()

0

**Same imputation values as previously!**

In [34]:
train_t[X_train['MasVnrArea'].isnull()]['MasVnrArea']

1278     99.765717
936      34.106592
650       0.000000
234     375.749332
973      85.817715
Name: MasVnrArea, dtype: float64

## Automatically find best imputation parameters

We can **optimise the parameters of the KNN imputation** to better predict our outcome.

**Import extra classes for modelling!**

In [35]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

**Separate intro train and test set!..**

In [36]:
X_train, X_test, y_train, y_test = train_test_split(
    data[cols_to_use],  # just the features
    data['SalePrice'],  # the target
    test_size=0.3,  # the percentage of obs in the test set
    random_state=0)  # for reproducibility
X_train.shape, X_test.shape

((1022, 36), (438, 36))

In [38]:
pipe = Pipeline(steps=[
    ('imputer', KNNImputer(
        n_neighbors=5,
        weights='distance',
        add_indicator=False)),
    ('scaler', StandardScaler()),
    ('regressor', Lasso(max_iter=2000)),])

**Create the grid with all the parameters that we would like to test!**

In [39]:
param_grid = {
    'imputer__n_neighbors': [3,5,10],
    'imputer__weights': ['uniform', 'distance'],
    'imputer__add_indicator': [True, False],
    'regressor__alpha': [10, 100, 200],}
grid_search = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1, scoring='r2')

**cv=3** is the cross-validation, **no_jobs =-1** indicates to use all available cpus, **scoring='r2'** indicates to evaluate using the r squared. **For more details in the grid parameters** visit: #https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

**Train over all the possible combinations of the parameters above!**

In [40]:
grid_search.fit(X_train, y_train)
print(("best linear regression from grid search: %.3f"
       % grid_search.score(X_train, y_train)))  # best score!

best linear regression from grid search: 0.845


**Check the performance over the test set print!**

In [41]:
print(("best linear regression from grid search: %.3f"
       % grid_search.score(X_test, y_test)))

best linear regression from grid search: 0.730


**Find the best parameters!**

In [42]:
grid_search.best_params_

{'imputer__add_indicator': True,
 'imputer__n_neighbors': 10,
 'imputer__weights': 'distance',
 'regressor__alpha': 200}

## Compare with univariate imputation

In [43]:
from sklearn.impute import SimpleImputer

**Separate intro train and test set!**

In [44]:
X_train, X_test, y_train, y_test = train_test_split(
    data[cols_to_use],  # just the features
    data['SalePrice'],  # the target
    test_size=0.3,  # the percentage of obs in the test set
    random_state=0)  # for reproducibility
X_train.shape, X_test.shape

((1022, 36), (438, 36))

**Train over all the possible combinations of the given parameters!**

In [45]:
pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean', fill_value=-1)),
    ('scaler', StandardScaler()),
    ('regressor', Lasso(max_iter=2000)),])
param_grid = {
    'imputer__strategy': ['mean', 'median', 'constant'],
    'imputer__add_indicator': [True, False],
    'regressor__alpha': [10, 100, 200],}
grid_search = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1, scoring='r2')
grid_search.fit(X_train, y_train)
print(("best linear regression from grid search: %.3f"
       % grid_search.score(X_train, y_train)))

best linear regression from grid search: 0.845


**Check the performance over the test set!**

In [46]:
print(("best linear regression from grid search: %.3f"
       % grid_search.score(X_test, y_test)))

best linear regression from grid search: 0.729


**Find the best fit parameters like this!**

In [47]:
grid_search.best_params_

{'imputer__add_indicator': False,
 'imputer__strategy': 'constant',
 'regressor__alpha': 200}

We see that imputing the values with an **arbitrary value of -1**, returns approximately the **same performance as doing KNN imputation**, so we might not want to add the additional complexity of training models to impute NA, too, then go ahead and predict the real target we are interested in.